<a href="https://colab.research.google.com/github/angelagomezM/TALENTOTECH/blob/main/PROYECTO_DETECCION_DE_Smishing_automatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Paso 1: Instalar librerías necesarias
!pip install -q ipywidgets

# Paso 2: Descargar dataset público y preparar datos
import pandas as pd

url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=["label", "message"])

# Convertir etiquetas 'ham' a 0 y 'spam' a 1
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

print(f"Dataset cargado con {len(df)} mensajes.")
print(df['label'].value_counts())

# Paso 3: Entrenar modelo de machine learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['message'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

print(f"Precisión en test: {modelo.score(X_test, y_test)*100:.2f}%")

# Paso 4: Función mejorada para detectar smishing con reglas y ML
import re

def detectar_sms_mejorado(texto):
    vector = vectorizer.transform([texto])
    pred = modelo.predict(vector)[0]

    # Detectar URLs sospechosas (simplificado)
    url_sospechosa = bool(re.search(r'http[s]?://[^\s]+', texto))
    palabras_peligro = ['bloqueada', 'actualice', 'suspensión', 'urgente', 'verifique', 'premio', 'clave', 'clic', 'reclama', 'evita', 'pago', 'alerta']

    # Si modelo dice smishing o si hay url sospechosa + palabra clave
    if pred == 1 or (url_sospechosa and any(p in texto.lower() for p in palabras_peligro)):
        mensaje = "⚠️ *Smishing detectado*"
        educacion = """
📚 **¿Qué hacer?**

- ❌ No hagas clic en enlaces sospechosos.
- 🔒 No compartas datos personales o bancarios.
- 🚫 Bloquea el número y repórtalo a tu operador móvil.
- 🛡️ Puedes reportar el mensaje a:
   • Policía Nacional: [https://caivirtual.policia.gov.co](https://caivirtual.policia.gov.co)
   • MinTIC: [https://www.osi.gov.co](https://www.osi.gov.co)
   • Correo: ciberseguridad@mintic.gov.co
        """
        return mensaje + "\n\n" + educacion
    else:
        return "✅ *Mensaje legítimo*"

# Paso 5: Interfaz interactiva para probar mensajes
import ipywidgets as widgets
from IPython.display import display, Markdown

entrada = widgets.Textarea(
    placeholder='Escribe aquí el mensaje SMS...',
    description='Mensaje:',
    layout=widgets.Layout(width='100%', height='100px')
)

boton = widgets.Button(description="Analizar", button_style='primary')
salida = widgets.Output()

def cuando_clic(b):
    with salida:
        salida.clear_output()
        resultado = detectar_sms_mejorado(entrada.value)
        display(Markdown("### Resultado:\n" + resultado))

boton.on_click(cuando_clic)

display(Markdown("## 📲 Detector de Smishing Mejorado con Ruta Educativa"))
display(entrada, boton, salida)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.5 MB/s eta 0:00:00
Dataset cargado con 5572 mensajes.
label
0    4825
1     747
Name: count, dtype: int64
Precisión en test: 96.41%


## 📲 Detector de Smishing Mejorado con Ruta Educativa

Textarea(value='', description='Mensaje:', layout=Layout(height='100px', width='100%'), placeholder='Escribe a…

Button(button_style='primary', description='Analizar', style=ButtonStyle())

Output()